# Send Encrypted Messages
## Using Public Key encryption
## Via twitter

Public-key, asymmetric security.

You need to setup Twitter OAuth 1.0a and get a;

Consumer key, Consumer secret, access_tok and access_token_sec

https://dev.twitter.com/oauth/application-only

### Getting Setup:

In [32]:
# First we need to make a Public, Private key pair. This can be slow
from pktwitter.key_tools import make_key_pair
Bob_key_pair = make_key_pair()
Alice_key_pair = make_key_pair()

Bob and Alice need to add their public key to the end of their twitter description (profile) so that others can send them messages.

Check out https://twitter.com/HeteroT1 as an example.

In [33]:
print('Bob needs this posted to be able to recieve a message.')
print(Bob_key_pair['PublicKey'] + '\n')

print('Alice needs to do the same with her public key')
print(Alice_key_pair['PublicKey'] + '\n')

# Bob and Alice might want to save there private keys to a file or store them somewhere safe. 
# The keys as a set of 3 long integers and a bit value
print("Bob's Private key " + str(Bob_key_pair['PrivateKey']) + '\n')
print("Alice's Private key " + str(Alice_key_pair['PrivateKey']))

Bob needs this posted to be able to recieve a message.
|TPK|ýHҔӢĺƸĆĘĜȜŁƖřǜҦψĚήƼǰĞÊĜķĜɕ|SӏɹÔȪӬȴöÈϴԳĘĊκǟϴɡԁϭƑӞƘëӁɛҀ|íɄҢĳχӿԉīҵȢʋΓƱʘжŀʌɁúĳŘäԫèɠՊ|ƍ

Alice needs to do the same with her public key
|TPK|äЅɹŔԪÞԴԙԗԚхκèϯɘȩԬҪԺΥʡѽϜǛƆƔ|ÁĂӤԟҸӭɤКƜέӁӿаХцĝåңjƚȮɇĬϻѾì|ÐԶљƕƸΩļƱͿĴӍɷӘιȕпŬҩĽoѩÂǿɷĀĢ|ƍ

Bob's Private key (112007804882173299121139143390180265204327822609141583302347145063143170143447, 18863483073404892414106061653962139125595330653459913646260878267095849453792, 53286256357958651503031828245325755124893030853022928006671142211559658595294, 256)

Alice's Private key (88669483199954082963937935938733595092648450403956826969574522789629326249263, 54117884943840893462690271582849911712701734144089819035269408433159659790096, 13688856604426814509611275635259973125320508092662214302334741603252942714275, 256)


### Twitter API key
You need a twitter api key. I load a set below you need your own. Here are instructios showing how to.
https://themepacific.com/how-to-generate-api-key-consumer-token-access-key-for-twitter-oauth/994/

In [23]:
import configparser
config = configparser.ConfigParser()
config.read('pktwitter/tests/user_data.ini')
consumer_key = config['HeteroT1']['consumer_key']
consumer_sec = config['HeteroT1']['consumer_sec']
access_tok = config['HeteroT1']['access_tok']
access_token_sec = config['HeteroT1']['access_token_sec']

First we need our twitter connection object.

In [24]:
from twython import Twython
bobs_twitter = Twython(consumer_key, consumer_sec, access_tok, access_token_sec)

## Sending a message
### Get Public Key
If Bob wants to send a message to Alice he only needs Alice's public key. Lets get that. (we are using the twitter account "HeteroT2" as Alice)
The key is compressed so we need to convert it to a key that can be used by our elgamal implementation.

In [25]:
from pktwitter.key_tools import get_public_key, assemble_publickey
Alice = 'HeteroT2'
alice_pub_comp = get_public_key(bobs_twitter, Alice)
print("Alice's cmpressed public key is: \n{}".format(alice_pub_comp))
print("You can see this on twitter at: https://twitter.com/heteroT2")

alice_pub = assemble_publickey(alice_pub_comp)
print("\nThe publickey used by elgamal is made using assemble_publickey()")
print("The public key object is alice_pub: {}".format(alice_pub))
print("The public key has the vlaues\np: {}\ng: {}\nh: {}\niNumBits: {}".format(alice_pub.p, alice_pub.g, alice_pub.h, alice_pub.iNumBits))

Alice's cmpressed public key is: 
|TPK|ËԈӦŏԨՌƍȲƃǍңҎƩϨҾðĵƹӣUԭԍъrĈŬ|oƫíȬԐöȜЁŴTjȉƮȌǊԢŏpưĩӵϠՓҪƺՍ|aȱόĠԯЮчјϾŀЋÁĺДїΥғԜԡӹŋʛïƧóȰ|ƍ
You can see this on twitter at: https://twitter.com/heteroT2

The publickey used by elgamal is made using assemble_publickey()
The public key object is alice_pub: <pktwitter.elgamal2.PublicKey object at 0x10497c630>
The public key has the vlaues
p: 64920886194952987256412246312819798284641846100168300883020957925738043123223
g: 40286097406928106390665231019035371289374310946194041291156901633994826301988
h: 26411613147959710735752662179675054173684751574803940945905190516099282103410
iNumBits: 256


## Encrypting the message
The message gets long when it is encrypted so you can't send 140 characters.

In [26]:
from pktwitter.messaging import encrypt_message
plaintext = "Hello Twitter"
cyphertext = encrypt_message(plaintext, alice_pub)
print("The cypertext is:\n{}".format(cyphertext))

The cypertext is:
HϷŁΚŽΛkԵĬϕՄŲńΎԲωϜĠѦҙӘԴҨȿĤʡ|PƌδҹǴͰϵóГǰԌηƘMΈǕҚrұɮѤŎǣԃǧԭ


## Sending a message
You may send as a direct message or as a status update. While the direct message route has the advantage of the recepient knowing the message is for them, and the data contained in the message may only be seen by the recepient, it also allows others to observe the metadata that there is direct interaction between the sender and recepient. With a public status update, there is no metadata with regards to whom the message is for. The recepient has to be expecting the message or just randomly try to decrypt messages. 

In [27]:
from pktwitter.messaging import send_direct_messages, send_status_update
from twython import Twython

bobs_twitter = Twython(consumer_key, consumer_sec, access_tok, access_token_sec)

send_direct_messages(bobs_twitter, 'HeteroT2', cyphertext)

send_status_update(bobs_twitter, cyphertext)

message sent 
message sent 
message -  HϷŁΚŽΛkԵĬϕՄŲńΎԲωϜĠѦҙӘԴҨȿĤʡ|PƌδҹǴͰϵóГǰԌηƘMΈǕҚrұɮѤŎǣԃǧԭ


## Reading a message
The simplest way to decrypt a messages is to copy the message from twitter.
Alica has stored her private key, this could be a ini file of just a text file.

In [28]:
from pktwitter.key_tools import assemble_privatekey
from pktwitter.messaging import decrypt_message

twitter_cyphertext = "hɃՋӕɨġȺͻѼҡɯѦiФԒʘQEÍЀђcƬҺϐӇ|WʨÅƷɠйάԶΈťӃıӞЍȟɗРҮђϲİӗĜƈŧβ"

alice_key_file = (64920886194952987256412246312819798284641846100168300883020957925738043123223,
                  40286097406928106390665231019035371289374310946194041291156901633994826301988,
                  12451816743310384350839455290582814769468821404604129753069361124628871636399,
                  256)

alice_private = assemble_privatekey(alice_key_file)

decryptedtext = decrypt_message(alice_private, twitter_cyphertext)
print("The decrypted text is:\n{}".format(decryptedtext))

UnicodeDecodeError: 'utf-16-le' codec can't decode bytes in position 28-29: illegal UTF-16 surrogate

In [29]:
from pktwitter.key_tools import key_compress
piv = (64920886194952987256412246312819798284641846100168300883020957925738043123223,
                         40286097406928106390665231019035371289374310946194041291156901633994826301988,
                         26411613147959710735752662179675054173684751574803940945905190516099282103410,
                         256
                         )
'|TPK|' + '|'.join(key_compress(n) for n in priv)

'|TPK|ËԈӦŏԨՌƍȲƃǍңҎƩϨҾðĵƹӣUԭԍъrĈŬ|oƫíȬԐöȜЁŴTjȉƮȌǊԢŏpưĩӵϠՓҪƺՍ|aȱόĠԯЮчјϾŀЋÁĺДїΥғԜԡӹŋʛïƧóȰ|ƍ'

In [ ]:
|TPK|ËԈӦŏԨՌƍȲƃǍңҎƩϨҾðĵƹӣUԭԍъrĈŬ|oƫíȬԐöȜЁŴTjȉƮȌǊԢŏpưĩӵϠՓҪƺՍ|aȱόĠԯЮчјϾŀЋÁĺДїΥғԜԡӹŋʛïƧóȰ|ƍ